# **Fera Formidável 4.1** - Quem classifica a classe classificadora?
**Objetivo:** altere a rede neural feita em Python puro para resolver um problema de
classificação. Treine uma rede neural em um dataset simples de classificação para mostrar
que funciona

**Autores:**

- Caio Matheus Leão Dantas
- Rafael Anis Shaikhzadeh Santos

**Contribuição:** Ambos discutiram o problema e fizeram o código todo juntos.

## **INTRODUÇÃO**

A atividade desenvolvida nesse notebook se baseia na aplicação e adaptação de uma rede neural para processo de classificação e utilização em um dataset.

O dataset "penguins" escolhido foi importado pela biblioteca ***seaborn*** e foi utilizado para treino da rede neural para o intuito de classificar os pinguins de forma binária baseada no seu sexo em função das suas informações de seus bicos e nadadeiras.

---

## **CÓDIGOS**

### ***Bibliotecas Importadas***

In [ ]:
from graphviz import Digraph
import pandas as pd
import seaborn as sns
import math
import random

## Funções importadas do material de aula

### *Função Grafo*

In [ ]:
def _tracar(folha):
    """Função modificada da criada por Andrej Karpathy para construção de grafo.

    Referência: https://github.com/karpathy/micrograd

    """
    vertices = set()
    arestas = set()

    def construir(v):
        """Função recursiva para traçar o grafo."""
        if v not in vertices:
            vertices.add(v)
            for progenitor in v.progenitor:
                arestas.add((progenitor, v))
                construir(progenitor)

    construir(folha)

    return vertices, arestas


def plota_grafo(folha):
    """Função modificada da criada por Andrej Karpathy para construção de grafo.

    Referência: https://github.com/karpathy/micrograd

    """
    grafo = Digraph(format="svg", graph_attr={"rankdir": "LR"})
    vertices, arestas = _tracar(folha)

    for v in vertices:
        id_vertice = str(id(v))

        if hasattr(v, "rotulo") and (hasattr(v, "grad")):
            texto = "{ " + f"{v.rotulo} | data {v.data:.3f} | grad {v.grad:.3f}" + " }"

        elif hasattr(v, "rotulo"):
            texto = "{ " + f"{v.rotulo} | data {v.data:.3f}" + " }"

        else:
            texto = "{ " + f"data {v.data:.3f}" + " }"

        grafo.node(name=id_vertice, label=texto, shape="record")

        if v.operador_mae:
            grafo.node(name=id_vertice + v.operador_mae, label=v.operador_mae)
            grafo.edge(id_vertice + v.operador_mae, id_vertice)

    for vertice1, vertice2 in arestas:
        grafo.edge(str(id(vertice1)), str(id(vertice2)) + vertice2.operador_mae)

    return grafo

### *Classe Valor*

Aqui, adicionamos alguns métodos a classe Valor vista em aula, para suprir futuras necessidades. Adiconamos os métodos dunders *\_\_rsub\_\_* e *\_\_gt\_\_*(maior que), além do método *log* (que utiliza math.log).

In [ ]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""

        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)

        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)

        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad

        resultado.propagar = propagar_adicao

        return resultado

    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""

        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)

        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)

        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data

        resultado.propagar = propagar_multiplicacao

        return resultado

    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)

        def propagar_exp():
            self.grad += resultado.grad * data

        resultado.propagar = propagar_exp

        return resultado

    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)

        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))

        resultado.propagar = propagar_pow

        return resultado

    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)

    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1

    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)

    def __rsub__(self, outro_valor):
        """Realiza a operação: outro_valor - self"""
        if not isinstance(outro_valor, Valor):
          outro_valor = Valor(outro_valor)

        return outro_valor - self

    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor

    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor

    def sig(self):
        """Realiza a operação: exp(self) / (exp(self) + 1)"""
        return self.exp() / (self.exp() + 1)

    def log(self):
        """Realiza a operação: log(self)"""
        progenitor = (self, )
        data = math.log10(self.data)
        operador_mae = "log"
        resultado = Valor(data, progenitor, operador_mae)

        def propagar_log():
            self.grad += resultado.grad * (self.data * math.log(10))**-1

        resultado.propagar = propagar_log

        return resultado

    def __gt__(self, outro_valor):
      """Realiza a operação: self > outro_valor."""

      if not isinstance(outro_valor, Valor):
          outro_valor = Valor(outro_valor)
      if self.data > outro_valor.data:
        data = True
      else:
        data = False
      progenitor = (self, outro_valor)
      operador_mae = ">"
      resultado = Valor(data, progenitor, operador_mae)
      return data

    def propagar(self):
        pass

    def propagar_tudo(self):

        self.grad = 1

        ordem_topologica = []

        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)

        for vertice in reversed(ordem_topologica):
            vertice.propagar()

### *Classes para MLP*

In [ ]:
class Neuronio:
    def __init__(self, num_dados_entrada):
        self.vies = Valor(random.uniform(-1, 1))

        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))

    def __call__(self, x):

        assert len(x) == len(self.pesos)

        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno

        soma += self.vies
        dado_de_saida = soma.sig()

        return dado_de_saida

    def parametros(self):
        return self.pesos + [self.vies]

In [ ]:
class Camada:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []

        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada)
            neuronios.append(neuronio)

        self.neuronios = neuronios

    def __call__(self, x):
        dados_de_saida = []

        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)

        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:
            return dados_de_saida

    def parametros(self):
        params = []

        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)

        return params

In [ ]:
class MLP:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):

        percurso = [num_dados_entrada] + num_neuronios_por_camada

        camadas = []

        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)

        self.camadas = camadas

    def __call__(self, x):
        for camada in self.camadas:
            x = camada(x)

        return x

    def parametros(self):
        params = []

        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)

        return params

### *Definição dos dados utilizados*

Aqui realizamos o processo de importação dos dados que vamos utilizar durante o treinamento da rede.

In [ ]:
df = sns.load_dataset("penguins")
df = df.dropna()
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


### *Transformação do sexo para valores binários*

Para o processo de classificação desejado na rede, é necessário a transformação dos valores binários em *string* de sexo dos pinguins - "Male" e "Female" - para dados binários numerais como 0 e 1. Nesse caso, foi definido o valor de 1 para os pinguins do sexo masculino e de 0 para o sexo feminino.

In [ ]:
for i in df["sex"]:
    if i == "Male":
        df["sex"] = df["sex"].replace(i, 1)
    else:
        df["sex"] = df["sex"].replace(i, 0)
df

<ipython-input-8-17c8032db724>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["sex"] = df["sex"].replace(i, 0)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,1
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,0
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,0
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,0
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,1
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,0
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,0
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,1
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,0


In [ ]:
len (df)

333

In [ ]:
count = df[df["sex"]==1]
count

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,1
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,1
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,1
13,Adelie,Torgersen,38.6,21.2,191.0,3800.0,1
14,Adelie,Torgersen,34.6,21.1,198.0,4400.0,1
...,...,...,...,...,...,...,...
333,Gentoo,Biscoe,51.5,16.3,230.0,5500.0,1
335,Gentoo,Biscoe,55.1,16.0,230.0,5850.0,1
337,Gentoo,Biscoe,48.8,16.2,222.0,6000.0,1
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,1


### *Definição dos atributos e target*

Definimos, portanto, as características que iremos analisar como atributos para previsão com nossa rede e o target do modelo já definido anteriomente.
Visto que é um exemplo didático, vamos resumir o df para 10 dados somente.

In [ ]:
X = [
    "bill_length_mm",
    "bill_depth_mm",
]
y = ["sex"]

df = df.reindex(X + y, axis=1)
df = df.head(10)
X = df.reindex(X, axis=1).values
y = df.reindex(y, axis=1).values.ravel()
df

,bill_length_mm,bill_depth_mm,sex
0,39.1,18.7,1
1,39.5,17.4,0
2,40.3,18.0,0
4,36.7,19.3,0
5,39.3,20.6,1
6,38.9,17.8,0
7,39.2,19.6,1
12,41.1,17.6,0
13,38.6,21.2,1
14,34.6,21.1,1


### *Criação da rede MLP*

Já nesse proceso, criamos a rede MLP própria para a classificação, definindo a quantidade de dados de entrada e o número de neurônios associado a cada camada.

Assim, fazemos a análise do sexo do pinguim a partir do valor obtido na previsão e o printamos para que a visualização em rótulo seja possível.
O valor do limiar escolhido é 0.5, ou seja, para valores acima de 0.5 consideramos o penguim macho (classe 1), e para os valores abaixo, consideramos como fêmea.


In [ ]:
dados_de_entrada = 2
num_neuronios_por_camada = [4, 3, 1]

minha_mlp = MLP(2, num_neuronios_por_camada)

y_pred = []

for x in X:
    previsao = minha_mlp(x)
    y_pred.append(previsao)

print(f"Os valores preditos para y são: {y_pred}")
print ()
class_pred = []
for pred in y_pred:
  if pred > 0.5:
    class_pred.append(1)
  else:
    class_pred.append(0)

print(f"Portanto, considerando limiar = 0.5, os rótulos preditos são: {class_pred}")

# for val in class_pred: #printar explicitamente se é macho ou fêmea
#   if val == 1:
#     print("Male")
#   else:
#     print("Female")

Os valores preditos para y são: [Valor(data=0.4243169320320481), Valor(data=0.4243169319342089), Valor(data=0.42431693219354705), Valor(data=0.42431693138127985), Valor(data=0.4243169322953025), Valor(data=0.4243169318369922), Valor(data=0.4243169321704952), Valor(data=0.4243169322891543), Valor(data=0.42431693223787326), Valor(data=0.4243169308453244)]

Portanto, considerando limiar = 0.5, os rótulos preditos são: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Todas predições foram de classe 1, isto é foi previsto que todos penguins são machos. Sabemos que esse resultado não reflete a realidade. Vamos então tentar melhorar a perfomance do nosso modelo, para isso vamos usar a função de perda Cross Entropy.

###  *Cálculo da função de perda (Cross Entropy)*

A partir da previsão e do valor real, precisamos analisar e calcular a função de perda da rede neural. No caso dessa rede, como envolve classificação, a função de perda foi a de Cross Entropy, de modo a se obter um valor que indica a perda associada a essa rede.

Primeiro, visto que estamos trabalhando com valores, devemos converter os valores reais de y em Classe Valor - inicialmente eles eram numpy, visto que vieram do dataset do seaborn.

In [ ]:
y_valor = [Valor(val) for val in y]


Podemos então calcular a função de perda por cross entropy com o código, baseado na equação da referência [2].

In [ ]:
erros = []

for yt, yp in zip(y_valor, y_pred):
  yp2 = 1-yp
  residuo = - (yt * yp.log() + (1 - yt) * yp2.log())
  erros.append(residuo)

loss = sum(erros)/len(erros)
loss

Valor(data=0.30606309080319777)

###  *Épocas*

E como último passo, realizamos o teste da rede baseado em seu aprendizado em várias épocas, treinando o modelo e visando a um valor decrescente do erro após cada época.

In [ ]:
NUM_EPOCAS = 10
TAXA_DE_APRENDIZADO = 0.001

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in X:
        previsao = minha_mlp(exemplo)
        y_pred.append(previsao)

    # loss
    erros = []

    for yt, yp in zip(y_valor, y_pred):
      residuo = - (yt * yp.log() + (1 - yt) * yp2.log())
      erros.append(residuo)

    loss = sum(erros)/len(erros)

    # zero grad
    for p in minha_mlp.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in minha_mlp.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

    # mostra resultado (opcional)
    print(epoca, loss.data)

0 0.3060630903951767
1 0.30605475835129614
2 0.30616770570011576
3 0.3066255599687096
4 0.3076739111709474
5 0.30884215640987234
6 0.3064732053395
7 0.2885937611343421
8 0.23733339318829474
9 0.17235794733874976


Vemos que a função de perda diminuiu, o que parece ser promissor. Mas, por fim, que tal analisar como está nossa predição agora.

In [ ]:
y_pred = []

for x in X:
    previsao = minha_mlp(x)
    y_pred.append(previsao)

print(f"Os valores preditos para y são: {y_pred}")
print ()

class_pred = []
for pred in y_pred:
  if pred > 0.5:
    class_pred.append(1)
  else:
    class_pred.append(0)

print(f"Portanto, considerando limiar = 0.5, os rótulos preditos são: {class_pred}")

Os valores preditos para y são: [Valor(data=0.9568164783617261), Valor(data=0.9568164790562224), Valor(data=0.9568164773919645), Valor(data=0.9568164821767626), Valor(data=0.9568164766341615), Valor(data=0.9568164796353998), Valor(data=0.9568164774524299), Valor(data=0.9568164767986317), Valor(data=0.9568164769364604), Valor(data=0.9568164845389469)]

Portanto, considerando limiar = 0.5, os rótulos preditos são: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


---

## **CONCLUSÃO**

Assim, conseguimos criar um classificador em python puro com sucesso! Ele consegue calcular valores entre 0 e 1, e a partir de um limiar, discriminar as classes.
Além do mais, aprendemos sobre a função de perda Cross Entropy para classificação binária, a qual foi essencial para treinarmos nosso modelo.

Porém, vemos que mesmo antes e depois do treino, a previsão é de que todos são machos, apesar de apenas metade do df ser classe 1. Isso mostrou, que apesar da função perda está diminuindo, não se houve melhora na acurácia... Curioso... Mas compreensível visto que é um exemplo didático.

Para o futuro, poderíamos tentar técnicas diferentes, como usar mais dados ou outra função de perda.

---

## **REFERÊNCIAS**

**[1]** CASSAR, Daniel. Redes Neurais e Algoritmos Genéticos. 2025. Material de Aula.

**[2]** HUGHES, Chris. A brief overview of cross entropy loss - Chris Hughes. Medium. 2024. https://medium.com/@chris.p.hughes10/a-brief-overview-of-cross-entropy-loss-523aa56b75d5

